In [6]:
import pandas as pd

In [29]:
results = pd.read_csv("results.txt")
results["pairid"] = results["pairid"].str[4:]
results["pairid"] = results["pairid"].astype(int)
results["CorrectACauseB"] = results["CorrectACauseB"].astype(float)
results["CorrectBCauseA"] = results["CorrectBCauseA"].astype(float)

In [30]:
# Computing accuracy
# The first column denotes whether chatGPT answered correctly for the question, "Does A cause B?"
# The second column denotes whether chatGPT answered correctly for the question, "Does B cause A?".
# if the result is 0,1 or 1,0, then we can take a random guess to decide the direction of causation. 
# Therefore, to compute accuracy, we can take the mean of "Does A cause B" and "Does B cause A" questions. 
results["accuracy"] = (results["CorrectACauseB"] + results["CorrectBCauseA"])/2

In [31]:
weights = pd.read_table("pairmeta.txt", sep=" ", header=None, names=["pairid", "v1", "v2", "v3", "v4", "weight"])
weights

/Users/julius/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,pairid,v1,v2,v3,v4,weight
0,1,1,1,2,2,0.1660
1,2,1,1,2,2,0.1660
2,3,1,1,2,2,0.1670
3,4,1,1,2,2,0.1660
4,5,1,1,2,2,0.1430
5,6,1,1,2,2,0.1430
6,7,1,1,2,2,0.1430
7,8,1,1,2,2,0.1430
8,9,1,1,2,2,0.1430
9,10,1,1,2,2,0.1430


In [19]:
dat = pd.merge(results[["pairid", "accuracy"]], weights, on="pairid")
dat

,accuracy,pairid,v1,v2,v3,v4,weight


In [21]:
dat["score"] = dat["accuracy"] * dat["weight"]
display(dat)
print("Accuracy", sum(dat["accuracy"])/len(dat))
print("Weighted accuracy", sum(dat["score"])/sum(dat["weight"]))

NameError: name 'display' is not defined